In [22]:
from torch.utils.data import DataLoader
from torchvision import transforms
from dataset import TrainDataset, ValidationDataset

In [23]:
tf = transforms.Compose([transforms.ToTensor()])

In [24]:
train_ds = TrainDataset(transforms=tf, val_ratio=0.2)
val_ds = ValidationDataset(transforms=tf)

In [25]:
BATCH_SIZE = 64

dl = {'train': DataLoader(train_ds, batch_size=BATCH_SIZE), 
    'val': DataLoader(val_ds, batch_size=BATCH_SIZE)}